# THE CODE FOR WEBSCRAPING THE DATA FROM MYCAREERS FUTURE
> CREDITS TO __SAMSON LEE GONG HAO__ FOR SCRAPING THIS - JULY 2018

In [1]:
from bs4 import BeautifulSoup
import urllib #use request if throws ERROR 403
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np

## Quick approach of first level job details

In [8]:
salaries = []
companies = []
titles = []
locations = []
emp_type = []
seniority = []
cat = []
 
for i in range(0,7):
    ##############open careersfuture webpage and selenium the html#############
    driver = webdriver.Chrome(executable_path="C:/Users/Samson/Downloads/chromedriver_win32/chromedriver")
    driver.get("https://www.mycareersfuture.sg/search?search=data%20scientist&page={}".format(i)) 
    # Wait one second.
    sleep(5)
    html = BeautifulSoup(driver.page_source, 'lxml')
    ##############iterate through job cards and get relevant details#############
    for job_card in html.find_all('div', id=lambda x: x and x.startswith('job-card-')):
        salaries.append(job_card.find_all('span', {'class':'lh-solid'}))
        companies.append(job_card.find_all('p', {'name':'company'}))
        titles.append(job_card.find_all('h1', {'name':'job_title'}))    
        locations.append(job_card.find_all('p', {'name':'location'}))
        emp_type.append(job_card.find_all('p', {'name':'employment_type'}))
        seniority.append(job_card.find_all('p', {'name':'seniority'}))
        cat.append(job_card.find_all('p', {'name':'category'}))
    driver.close()

In [9]:
#create dictionary and iterate through scraped list to append information
lists = [salaries, companies, titles, locations, emp_type, seniority, cat]
clean = {}
for ind, ls in enumerate(lists):
    clean[ind] = []
    for e in ls:
        try:
            clean[ind].append(e[0].text)
        except:
            clean[ind].append('NONE')
#convert dictionary into dataframe and rename columns
df = pd.DataFrame(clean)
df.columns = ['salaries', 'companies', 'titles', 'locations', 'emp_type', 'seniority', 'category']

In [10]:
len(df)

121

In [23]:
df.to_csv('salaries.csv', encoding='utf-8')

In [21]:
df

,salaries,companies,titles,locations,emp_type,seniority,category
0,Salary undisclosed,Company Undisclosed,Data Scientist,Central,Full Time,Professional,Information Technology
1,"$5,256to$7,008",UPS ASIA GROUP PTE. LTD.,APAC Data Scientist,East,Permanent,Manager,Logistics / Supply Chain
2,"$11,000to$20,500",AIRBNB SINGAPORE PRIVATE LIMITED,"Data Scientist - Analytics, Policy",Central,Full Time,Executive,Others
3,"$6,000to$7,000",RESMED ASIA PTE. LTD.,Data Scientist,South,Permanent ...,Executive,Engineering ...
4,"$7,482to$9,975",UPS ASIA GROUP PTE. LTD.,APAC Lead Data Scientist,East,Permanent,Middle Management,Logistics / Supply Chain
5,"$5,500to$6,500",RESMED ASIA OPERATIONS PTY LIMITED (SINGAPORE ...,Data Scientist,South,Permanent ...,Senior Executive,Healthcare / Pharmaceutical ...
6,"$6,000to$10,000",OAK CONSULTING PTE. LTD.,Data Scientist,East,Contract,Professional,Information Technology
7,"$4,500to$6,500",SENSE INFOSYS PTE. LTD.,Data Scientist,West,Full Time,Professional ...,Information Technology
8,"$6,000to$10,000",CERTIS CISCO SECURITY PTE. LTD.,Data Scientist,East,Full Time,Senior Executive,Information Technology
9,"$6,500to$13,000",ACCENTURE PTE LTD,Data Scientist,Islandwide,Permanent,Professional,Consulting


## Get job_urls of all related search

In [50]:
job_urls = []

In [96]:
driver = webdriver.Chrome(executable_path="C:/Users/Samson/Downloads/chromedriver_win32/chromedriver")
for i in range(0,4):
    #######this part is manually done: input the key words and page range######
    ##############open careersfuture webpage and selenium the html#############    
#     driver.get("https://www.mycareersfuture.sg/search?search=data%20analyst&page={}".format(i)) #search data analyst
#     driver.get("https://www.mycareersfuture.sg/search?search=data%20scientist&page={}".format(i)) #search data scientist
#     driver.get("https://www.mycareersfuture.sg/search?search=analytics&page={}".format(i)) #search analytics
#     driver.get("https://www.mycareersfuture.sg/search?search=machine%20learning&page={}".format(i)) #machine learning
#     driver.get("https://www.mycareersfuture.sg/search?search=business%20intelligence&page={}".format(i)) #business intelligence
#     driver.get("https://www.mycareersfuture.sg/search?search=business%20analyst&page={}".format(i)) #business analyst
    driver.get("https://www.mycareersfuture.sg/search?search=data%20engineer&page={}".format(i)) #data engineer

    # Wait one second.
    sleep(5)
    html = BeautifulSoup(driver.page_source, 'lxml')
    for job_card in html.find_all('div', id=lambda x: x and x.startswith('job-card-')):
        job_urls.append('https://www.mycareersfuture.sg'+job_card.find('a')['href'])

In [99]:
job_urls2 = set(job_urls)
len(job_urls2)

1628

In [100]:
#save urls as csv
pd.DataFrame(list(job_urls2)).to_csv('job_url2.csv', encoding='utf-8')

In [2]:
job_df = pd.read_csv('job_url2.csv')
job_ls = job_df.iloc[:,1].values

## Loop through each job_url to get details

In [32]:
companies = []
titles = []
addresses = []
emp_type = []
seniority = []
category = []
salaries = []
responsibilities = []
requirements = []
failed_url = []


driver = webdriver.Chrome(executable_path="C:/Users/Samson/Downloads/chromedriver_win32/chromedriver")
for ind, url_job in enumerate(job_ls[1008:1012]): #error at 1010, to use try except for future runs
    try:
        driver.get(url_job)
        sleep(5)
        html_job = driver.page_source
        soup_job = BeautifulSoup(html_job, 'lxml')
        companies.append(soup_job.find('p', {'id': 'company'}))
        titles.append(soup_job.find('h1', {'id': 'job_title'}))
        addresses.append(soup_job.find('p', {'id': 'address'}))    
        emp_type.append(soup_job.find('p', {'id': 'employment_type'}))
        seniority.append(soup_job.find('p', {'id': 'seniority'}))
        category.append(soup_job.find('p', {'id': 'job-categories'}))
        salaries.append(soup_job.find('div', {'class': 'lh-solid'}))
        responsibilities.append(soup_job.find('div',{'id':'job_description'}))
        requirements.append(soup_job.find('div',{'id':'requirements'}))
    except:
        failed_url.append(url_job)

In [31]:
#create dictionary and iterate through scraped list to append information
lists = [companies, titles, addresses, emp_type, seniority, category, salaries, responsibilities, requirements]
clean = {}
for ind, ls in enumerate(lists):
    clean[ind] = []
    for e in ls:
        try:
            clean[ind].append(e.text)
        except:
            clean[ind].append('NONE')
#convert dictionary into dataframe and rename columns
df = pd.DataFrame(clean)
df.columns = ['Company', 'Title', 'Address', 'Emp_type', 'Seniority', 'Industry', 'Salary', 'Responsibility', 'Requirements']
df

,Company,Title,Address,Emp_type,Seniority,Industry,Salary,Responsibility,Requirements
0,INTERCONTINENTAL HOTELS GROUP (ASIA PACIFIC) P...,"Manager, Sales Effectiveness - SEAK","BUGIS JUNCTION, 230 VICTORIA STREET 188024",Permanent,Manager,Sales / Retail,"$6,000to$11,000",Roles & ResponsibilitiesWhat's your passion? W...,RequirementsEducation – Bachelor's degree in B...
1,CLARIANT (SINGAPORE) PTE. LTD.,Data Scientist,"THE SYNERGY, 1 INTERNATIONAL BUSINESS PARK 609917","Permanent, Full Time","Middle Management, Manager, Professional","Manufacturing, Others","$5,000to$8,000",Roles & ResponsibilitiesResponsibilities: Per...,RequirementsRequirements: Master in chemistry...
2,CAPGEMINI SINGAPORE PTE. LTD.,.Net Developer (1-year contract),6 BATTERY ROAD 049909,Contract,Professional,Information Technology,"$4,000to$6,500",Roles & ResponsibilitiesObjective Align with ...,RequirementsQualification o Training Bachelo...


In [17]:
#save dataframe into csv
df.to_csv('salary_df.csv', encoding='utf-8')

In [37]:
salary_df = pd.read_csv('salary_df.csv')
salary_df.drop('Unnamed: 0', axis=1)

,Company,Title,Address,Emp_type,Seniority,Industry,Salary,Responsibility,Requirements
0,SKILLSFUTURE SINGAPORE AGENCY,"Executive, (Quality Management Division) (6-mo...","ONE MARINA BOULEVARD, 1 MARINA BOULEVARD 018989",Contract,NONE,Public / Civil Service,NONE,Roles & ResponsibilitiesResponsibilities\r\n\r...,NONE
1,HP PPS ASIA PACIFIC PTE. LTD.,Business Analyst,1 DEPOT CLOSE 109841,"Permanent, Full Time",Professional,Others,"$8,400to$12,900",Roles & ResponsibilitiesHP is the world’s lead...,RequirementsEducation and Experience Required:...
2,SHOPEE SINGAPORE PRIVATE LIMITED,Software Engineer,"GALAXIS, 1 FUSIONOPOLIS PLACE 138522","Permanent, Full Time",Executive,Information Technology,"$4,400to$8,000",Roles & ResponsibilitiesResponsibilities: De...,RequirementsRequirements: Minimum B.S. degre...
3,PRICEWATERHOUSECOOPERS CONSULTING (SINGAPORE) ...,Data Analytics –Manager,"MARINA ONE EAST TOWER, 7 STRAITS VIEW 018936","Permanent, Contract, Full Time",Manager,Consulting,"$6,200to$9,500",Roles & Responsibilities Advisory - Consulting...,Requirements A good Degree in a quantitative ...
4,Company Undisclosed,Research Fellow,NONE,"Contract, Full Time","Professional, Executive",Others,"$4,000to$5,000",Roles & ResponsibilitiesData Scientist / Progr...,Requirements Doctorate degree in a relevant fi...
5,Republic Polytechnic,Office of Student and Graduate Affairs - Admin...,9 WOODLANDS AVENUE 9 738964,Full Time,NONE,"Education and Training, Public / Civil Service",NONE,Roles & ResponsibilitiesJob Responsibilities T...,NONE
6,"JPMORGAN CHASE BANK, N.A.","Application Support Analyst, Reference Data Team",NONE,Full Time,Executive,Information Technology,"$6,000to$12,000",Roles & ResponsibilitiesWe are seeking an Appl...,RequirementsQualifications 7-10 years Product...
7,"JPMORGAN CHASE BANK, N.A.",CyberSecurity Business Resiliency Analyst and ...,NONE,Full Time,Executive,Information Technology,"$7,500to$15,000",Roles & ResponsibilitiesWorking in Cybersecuri...,Requirements• Bachelors’ degree in computer sc...
8,Company Undisclosed,Software Engineer (Ref 21711),NONE,"Contract, Full Time",Executive,Information Technology,"$2,500to$5,000",Roles & Responsibilities- Research and develop...,Requirements- Degree in Computer Science/Infor...
9,FINSURGE PTE. LTD.,Murex Technical Migration Consultant,"THE SIGNATURE, 51 CHANGI BUSINESS PARK CENTRAL...",Permanent,"Middle Management, Professional",Information Technology,"$4,000to$7,000",Roles & ResponsibilitiesFinSurge is undertakin...,RequirementsMust Have: Hands on experience i...


In [40]:
salary_df = salary_df.replace('NONE', np.nan)

In [41]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 10 columns):
Unnamed: 0        1627 non-null int64
Company           1469 non-null object
Title             1469 non-null object
Address           1089 non-null object
Emp_type          1469 non-null object
Seniority         1425 non-null object
Industry          1469 non-null object
Salary            1302 non-null object
Responsibility    1469 non-null object
Requirements      1398 non-null object
dtypes: int64(1), object(9)
memory usage: 127.2+ KB
